## Step05_kNearestNeighbor

k最近傍法で分類する
混同行列と各クラスの認識率を表示する

In [10]:
from skimage import io
import numpy as np

In [11]:
TrainingSampleNum = 2000 # 学習サンプル総数
TestSampleNum = 10000 # テストサンプル総数
ClassNum = 10 # クラス数（今回は10）
ImageSize = 28 # 画像サイズ（今回は縦横ともに28）
TrainingDataFile = '/home/share/students/Wataru_Ohyama/Images/TrainingSamples/{0:1d}-{1:04d}.png'
TestDataFile = '/home/share/students/Wataru_Ohyama/Images/TestSamples/{0:1d}-{1:04d}.png'
OutFile = './Images/OutSamples/save_{0:1d}-{1:04d}.png'

In [13]:
# LoadTemplates
def LoadTemplates () :
    labels = np.zeros( TrainingSampleNum, dtype=np.uint8 )
    templates = np.zeros( (TrainingSampleNum,ImageSize,ImageSize), dtype=np.uint8 )

    i = 0
    for label in range (0, ClassNum):
        for sample in range (0, TrainingSampleNum // ClassNum ):
            filename = TrainingDataFile.format(label,sample)
            templates[i,:,:] = io.imread(filename)
            labels[i]=label
            i += 1

    return templates, labels

In [4]:
# CalcDistance
# 今回は L1 距離を算出
def CalcDistance ( target, template ):
    buf = np.fabs( target - template )    
    return np.sum(buf)

In [5]:
# ReturnMatchLabel
def ReturnMatchLabel ( target, templates, labels, K ):
    min_dis = np.full(K, float("Inf"))
    k_labels = np.full(K, -1)
    
    for i in range (0, TrainingSampleNum ):
        dis = CalcDistance ( target, templates[i,:,:] )
        label = labels[i]
        for k in range(0,K):
            if min_dis[k] > dis :
                min_dis[k], dis = dis, min_dis[k]
                k_labels[k], label = label, k_labels[k]

    class_count = np.zeros(ClassNum)
    for k in range (0,K):
        class_count [k_labels[k]]+=1
        
    match_label = 0;
    max_count = 0;
    for i in range (0,ClassNum):
        if max_count < class_count[i]:
            max_count = class_count[i]
            match_label = i
    
    return match_label

In [6]:
# kNearestNeighbor
def kNearestNeighbor ( templates, labels, k ):

    results = np.zeros ( (ClassNum, ClassNum) )
    each_class_num = TestSampleNum // ClassNum
    
    for label in range (0, ClassNum):
        for sample in range (0, each_class_num ):
            filename = TestDataFile.format(label,sample)
            t_img = io.imread ( filename )
            match_label = ReturnMatchLabel ( t_img, templates, labels, k )
            results [label, match_label] += 1
        print ( "{0:1d}: {1:.4f}".format(label, results[label,label]/each_class_num))

    print("= Confusion matrix ===========")
    for t_label in range (0,ClassNum):

        for m_label in range (0,ClassNum):
            print ("{:04g}, ".format(results[t_label,m_label]), end="")
        print("")

    print("= Total Recognition accuracy ===========")
    total_correct_num = 0
    for t_label in range (0,ClassNum):
        total_correct_num += results[t_label,t_label]
    print ("TOTAL: {0:.4f}".format(total_correct_num / TestSampleNum))


In [7]:
# main ルーチン

templates, labels = LoadTemplates ()
k = 3
kNearestNeighbor ( templates, labels, k )


0: 0.9060
1: 0.9940
2: 0.6190
3: 0.6090
4: 0.5520
5: 0.3990
6: 0.7850
7: 0.7000
8: 0.4040
9: 0.6450
= Confusion matrix ===========
0906, 0040, 0001, 0001, 0001, 0008, 0038, 0001, 0003, 0001, 
0000, 0994, 0003, 0000, 0000, 0000, 0003, 0000, 0000, 0000, 
0105, 0202, 0619, 0015, 0006, 0001, 0015, 0013, 0015, 0009, 
0030, 0239, 0030, 0609, 0002, 0019, 0010, 0010, 0027, 0024, 
0107, 0171, 0008, 0004, 0552, 0001, 0021, 0016, 0003, 0117, 
0104, 0260, 0007, 0087, 0013, 0399, 0070, 0001, 0026, 0033, 
0061, 0140, 0006, 0000, 0001, 0007, 0785, 0000, 0000, 0000, 
0033, 0159, 0011, 0002, 0013, 0001, 0001, 0700, 0001, 0079, 
0035, 0368, 0039, 0038, 0007, 0036, 0043, 0010, 0404, 0020, 
0044, 0155, 0002, 0014, 0069, 0005, 0003, 0060, 0003, 0645, 
= Total Recognition accuracy ===========
TOTAL: 0.6613
